# **Install and import what is necessary to carry out the training**

In [ ]:
# Importing libraries and mounting the drive where the dataset is stored
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style(style = 'whitegrid')
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install datasets
!pip install transformers==4.28.0  # Later versions are having problems with TrainingArguments.
!pip install evaluate
!pip install wandb
import tensorflow as tf
import gc, torch
from datasets import load_dataset
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback, DataCollatorWithPadding, TextClassificationPipeline, logging
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix


# **Define common training functions to all models**

In [ ]:
def load_corpus_Salud(drop, n_labels, task):
  dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DatasetMH_Emotions.csv", sep=";")[["Text", "Polarity", "Emotions"]]
  dataset.drop(dataset.tail(drop).index, inplace = True)
  dataset.columns = ["text", "label", "emotion"]
  if(task=="sentiment"):
    if(n_labels==2):
      dataset = dataset.loc[dataset["label"] != "Indeterminado"]
      dataset['label'] = [0 if item=="Negativa" else 1 for item in dataset['label']]
    else:
      dataset['label'] = [0 if item=="Negativa" else 1 if item=="Indeterminado" else 2 for item in dataset['label']]
    return dataset[["text", "label"]]
  else:
    dataset['label'] = [0 if item=="Amor/Admiración" else 1 if item=="Gratitud" else 2 if item=='Tristeza/Pena' else 3 if item=='Enfado/Desprecio/Burla' else 4 if item=="Comprensión/Empatía/Identificación"
                          else 5 for item in dataset['emotion']]
    return dataset[["text", "label"]]    # Mandatory for headers to be ["text", "label"] for trainer
      
#Label Texto is Text
#Label Emociones is Emotions

#Label Positiva is Positive
#Label Negativa is Negative
#Label Indeterminado is Neutral

#Label Amor/Admiración is Love/admiration
#Label Gratitud is Gratitude
#Label Tristeza/Pena is Sadness
#Label Enfado/Desprecio/Burla is Anger/contempt/mockery
#Label Comprensión/Empatía/Identificación is Comprehension/empathy/identification
#Label Indeterminado is Neutral

def compute_metrics(eval_pred):
    metric1 = evaluate.load("accuracy")
    metric2 = evaluate.load("precision")
    metric3 = evaluate.load("recall")
    metric4 = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    precision = metric2.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric3.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric4.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy":accuracy, "precision":precision, "recall":recall, "f1":f1}


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def garbage_collector():
  torch.cuda.empty_cache()
  gc.collect()
  !nvidia-smi

earlyStop=EarlyStoppingCallback()


#Prediction function
#name = "roberta", "robertuito" if task=="sentiment". "daveni", "robertuito" if task=="emotion"
#task = "sentiment", "emotion"
#data = string data or list of strings to predict from

#return = prediction or list of predictions
def predict(name, task, data):
  if(task=="emotion"):
    num_labels=6
    id2label = {0: "Amor/Admiración", 1: "Gratitud", 2: "Tristeza/Pena",
                3:"Enfado/Desprecio/Burla", 4:"Comprensión/Empatía/Identificación", 5:"Indiferente"}
    label2id = {"Amor/Admiración":0, "Gratitud":1, "Tristeza/Pena":2,
                "Enfado/Desprecio/Burla":3, "Comprensión/Empatía/Identificación":4, "Indiferente":5}
  else:
    num_labels=3
    id2label = {0: "NEG", 1: "NEU", 2: "POS"}
    label2id = {"NEG": 0, "NEU": 1, "POS": 2}

  logging.set_verbosity_error()
  model_path = "./drive/MyDrive/Colab Notebooks/health/model_"+name+"_"+task
  tok_path = "./drive/MyDrive/Colab Notebooks/health/tok_"+name+"_"+task
  model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels,
                              label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
  tokenizer = AutoTokenizer.from_pretrained(tok_path)
  pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=1, device=0)
  logging.set_verbosity_warning()
  return([i[0]["label"] for i in pipe(data)])


# **Analysis of Emotions**

In [ ]:
#Label Amor/Admiración is Love/admiration
#Label Gratitud is Gratitude
#Label Tristeza/Pena is Sadness
#Label Enfado/Desprecio/Burla is Anger/contempt/mockery
#Label Comprensión/Empatía/Identificación is Comprehension/empathy/identification
#Label Indeterminado is Neutral

dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DatasetMH_Emotions.csv", sep=";")[["Text", "Emotions"]]

print(dataset.head())

plt.figure(figsize=(17, 3))
sns.countplot(x = 'Emotions', data = dataset, palette = 'rocket',
              order=['Amor/Admiración', 'Gratitud', 'Tristeza/Pena', 'Enfado/Desprecio/Burla', 'Comprensión/Empatía/Identificación', 'Indeterminado']);

In [ ]:
#Label Amor/Admiración is Love/admiration
#Label Gratitud is Gratitude
#Label Tristeza/Pena is Sadness
#Label Enfado/Desprecio/Burla is Anger/contempt/mockery
#Label Comprensión/Empatía/Identificación is Comprehension/empathy/identification
#Label Indeterminado is Neutral

print(len(dataset[dataset["Emociones"]=='Amor/Admiración']))
print(len(dataset[dataset["Emociones"]=='Gratitud']))
print(len(dataset[dataset["Emociones"]=='Tristeza/Pena']))
print(len(dataset[dataset["Emociones"]=='Enfado/Desprecio/Burla']))
print(len(dataset[dataset["Emociones"]=='Comprensión/Empatía/Identificación']))
print(len(dataset[dataset["Emociones"]=='Indeterminado']))

## **Daveni Model**

##### **Initial test**

In [ ]:
dataset = load_corpus_Salud(0, n_labels=6, task="emotion")
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}


tokenizer = AutoTokenizer.from_pretrained("daveni/twitter-xlm-roberta-emotion-es")
model = AutoModelForSequenceClassification.from_pretrained("daveni/twitter-xlm-roberta-emotion-es", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
data_collator = DataCollatorWithPadding(tokenizer)

train, test = train_test_split(dataset, stratify=dataset["label"])

train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

training_args = TrainingArguments(num_train_epochs = 10, output_dir="best-sweep-12", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #necesario para earlyStop

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[earlyStop]
  )
trainer.train()
pred=trainer.predict(tokenized_datasets["test"])
logits=pred.predictions
prob=tf.nn.softmax(logits)
res=np.argmax(prob, axis=1) # predictions

In [ ]:
print(classification_report(tokenized_datasets["test"]["label"], res, digits=4))

##### **Tunning process**

In [ ]:
import wandb
wandb.login()

In [ ]:
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}

tokenizer = AutoTokenizer.from_pretrained("daveni/twitter-xlm-roberta-emotion-es")
model = AutoModelForSequenceClassification.from_pretrained("daveni/twitter-xlm-roberta-emotion-es", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
data_collator = DataCollatorWithPadding(tokenizer)


dataset = load_corpus_Salud(drop=0, n_labels = 6, task="emotion") # with emotion the number of labels does not matter
train, test = train_test_split(dataset, stratify=dataset["label"])
train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

In [ ]:
config = {
          "wandb":{
            "project": "wandb-daveni-emotion",
            "api_key": "XXXXXX"
        }
}


def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [2, 4, 8, 16, 32, 64, 128]},
            "per_device_eval_batch_size": {"values": [2, 4, 8, 16, 32, 64, 128]},
            "num_train_epochs" : {"distribution": "int_uniform", "min": 5, "max": 15}
        },
    }

def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
        "daveni/twitter-xlm-roberta-emotion-es", ignore_mismatched_sizes=True,
        config=config, num_labels=6, label2id = label2id, id2label=id2label
    )


training_args = TrainingArguments(report_to="wandb", output_dir="wandb-daveni-emotion", overwrite_output_dir=True,
       evaluation_strategy="epoch", per_device_train_batch_size = 128, per_device_eval_batch_size=128, num_train_epochs = 10,
                                    fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") # necessary to earlyStop

trainer = Trainer(
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
    data_collator=data_collator,
    callbacks=[earlyStop]
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=30
)


wandb.finish()

##### **KFold process**

In [ ]:
dataset = load_corpus_Salud(0, n_labels=6, task="emotion")
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}
splits=10
JOY = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
FEAR = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
SAD = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
ANG = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
SUR = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
OTHER = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])

macro = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
weighted = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
acc= pd.DataFrame(columns=["accuracy"])

i=0
kf = StratifiedKFold(n_splits = splits, random_state=10, shuffle=True)
for train, test in kf.split(X=dataset["text"], y=dataset["label"]):
  print("\n\n******************LOOP", i, "**************************\n\n")

  tokenizer = AutoTokenizer.from_pretrained("daveni/twitter-xlm-roberta-emotion-es")
  model = AutoModelForSequenceClassification.from_pretrained("daveni/twitter-xlm-roberta-emotion-es", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
  data_collator = DataCollatorWithPadding(tokenizer)
  dataset.iloc[train].to_csv("corpus_train.csv", index=False)
  dataset.iloc[test].to_csv("corpus_test.csv", index=False)
  train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
  tokenized_datasets = train_test.map(tokenize_function, batched=True)

  training_args = TrainingArguments(learning_rate=0.00001409, num_train_epochs=12, per_device_train_batch_size = 8, per_device_eval_batch_size=2,
                                  output_dir="best-sweep-2", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #to earlyStop

  trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[earlyStop]
  )
  trainer.train()
  pred=trainer.predict(tokenized_datasets["test"])
  logits=pred.predictions
  prob=tf.nn.softmax(logits)
  res=np.argmax(prob, axis=1) # predictions

  report = classification_report(tokenized_datasets["test"]["label"], res, output_dict=True)
  JOY.loc[i] = report["0"]
  FEAR.loc[i] = report["1"]
  SAD.loc[i] = report["2"]
  ANG.loc[i] = report["3"]
  SUR.loc[i] = report["4"]
  OTHER.loc[i] = report["5"]
  macro.loc[i] = report["macro avg"]
  weighted.loc[i] = report["weighted avg"]
  acc.loc[i] = report["accuracy"]
  i+=1






In [ ]:
JOY

In [ ]:
print(np.mean(JOY, axis=0))
# "Joy" translates in our dataset as the emotion "Love/Admiration"

In [ ]:
FEAR

In [ ]:
print(np.mean(FEAR, axis=0))
# "FEAR" translates in our dataset as the emotion "Gratitude"

In [ ]:
SAD

In [ ]:
print(np.mean(SAD, axis=0))
# "SAD" translates in our dataset as the emotion "Sadness"

In [ ]:
ANG

In [ ]:
print(np.mean(ANG, axis=0))
# "ANG" translates in our dataset as the emotion "Anger/Contempt/Mockery"

In [ ]:
SUR

In [ ]:
print(np.mean(SUR, axis=0))
# "SUR" translates in our dataset as the emotion "Comprehension/Empathy/Identification"

In [ ]:
OTHER

In [ ]:
print(np.mean(OTHER, axis=0))
# "OTHER" translates in our dataset as the emotion "Neutral"

In [ ]:
macro

In [ ]:
print(np.mean(macro, axis=0))
# macro metric

In [ ]:
weighted

In [ ]:
print(np.mean(weighted, axis=0))
# weighted metric

In [ ]:
acc

In [ ]:
print(np.mean(acc, axis=0))
# accuracy metric (global metric)

## **ROBERTUITO Model**

##### Initial test

In [ ]:
dataset = load_corpus_Salud(0, n_labels=6, task="emotion")
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}


tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-emotion-analysis", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
data_collator = DataCollatorWithPadding(tokenizer)

train, test = train_test_split(dataset, stratify=dataset["label"])

train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

training_args = TrainingArguments(output_dir="try", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #for earlyStop

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[earlyStop]
  )
trainer.train()
pred=trainer.predict(tokenized_datasets["test"])
logits=pred.predictions
prob=tf.nn.softmax(logits)
res=np.argmax(prob, axis=1) # predictions
print(classification_report(tokenized_datasets["test"]["label"], res, digits=4))

**Tunning process of the model**

In [ ]:
import wandb
wandb.login()

In [ ]:
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}

tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-emotion-analysis", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
data_collator = DataCollatorWithPadding(tokenizer)


dataset = load_corpus_Salud(drop=0, n_labels = 6, task="emotion") 
train, test = train_test_split(dataset, stratify=dataset["label"])
train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

In [ ]:
config = {
          "wandb":{
            "project": "wandb-robertuito-emotion",
            "api_key": "XXXXXXXXXX"
        }
}


def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [2, 4, 8, 16, 32, 64]},
            "per_device_eval_batch_size": {"values": [2, 4, 8, 16, 32, 64]},
            "num_train_epochs" : {"distribution": "int_uniform", "min": 5, "max": 15}
        },
    }

def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
        "pysentimiento/robertuito-emotion-analysis", ignore_mismatched_sizes=True,
        config=config, num_labels=6, label2id = label2id, id2label=id2label
    )


training_args = TrainingArguments(report_to="wandb" , output_dir="wandb-robertuito-emotion", overwrite_output_dir=True,
       evaluation_strategy="epoch", per_device_train_batch_size = 128, per_device_eval_batch_size=128, num_train_epochs = 10,
                                    fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") # Necessary for earlyStop

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
    data_collator=data_collator,
    callbacks=[earlyStop]
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=15
)


wandb.finish()

##### **KFold process**

In [ ]:
dataset = load_corpus_Salud(0, n_labels=6, task="emotion")
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}


splits=10
JOY = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
FEAR = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
SAD = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
ANG = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
SUR = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
OTHER = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])


macro = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
weighted = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
acc= pd.DataFrame(columns=["accuracy"])

i=0
kf = StratifiedKFold(n_splits = splits, random_state=10, shuffle=True)
for train, test in kf.split(X=dataset["text"], y=dataset["label"]):
  print("\n\n******************LOOP", i, "**************************\n\n")

  tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")
  model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-emotion-analysis", num_labels=6, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True )
  data_collator = DataCollatorWithPadding(tokenizer)

  dataset.iloc[train].to_csv("corpus_train.csv", index=False)
  dataset.iloc[test].to_csv("corpus_test.csv", index=False)
  train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
  tokenized_datasets = train_test.map(tokenize_function, batched=True)

  training_args = TrainingArguments(learning_rate=0.00007557, num_train_epochs=6, per_device_train_batch_size = 16, per_device_eval_batch_size=16,
                                  output_dir="best-sweep-", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #for earlyStop

  trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[earlyStop]
  )
  trainer.train()
  pred=trainer.predict(tokenized_datasets["test"])
  logits=pred.predictions
  prob=tf.nn.softmax(logits)
  res=np.argmax(prob, axis=1) # predictions

  report = classification_report(tokenized_datasets["test"]["label"], res, output_dict=True)
  JOY.loc[i] = report["0"]
  FEAR.loc[i] = report["1"]
  SAD.loc[i] = report["2"]
  ANG.loc[i] = report["3"]
  SUR.loc[i] = report["4"]
  OTHER.loc[i] = report["5"]
  macro.loc[i] = report["macro avg"]
  weighted.loc[i] = report["weighted avg"]
  acc.loc[i] = report["accuracy"]
  i+=1



In [ ]:
JOY

In [ ]:
print(np.mean(JOY, axis=0))
# "Joy" translates in our dataset as the emotion "Love/Admiration"

In [ ]:
FEAR

In [ ]:
print(np.mean(FEAR, axis=0))
# "FEAR" translates in our dataset as the emotion "Gratitude"

In [ ]:
SAD

In [ ]:
print(np.mean(SAD, axis=0))
# "SAD" translates in our dataset as the emotion "Sadness"

In [ ]:
ANG

In [ ]:
print(np.mean(ANG, axis=0))
# "ANG" translates in our dataset as the emotion "Anger/Contempt/Mockery"

In [ ]:
SUR

In [ ]:
print(np.mean(SUR, axis=0))
# "SUR" translates in our dataset as the emotion "Comprehension/Empathy/Identification"

In [ ]:
OTHER


In [ ]:
print(np.mean(OTHER, axis=0))
# "OTHER" translates in our dataset as the emotion "Neutral"

In [ ]:
macro

In [ ]:
print(np.mean(macro, axis=0))
# macro metric

In [ ]:
weighted

In [ ]:
print(np.mean(weighted, axis=0))
# weighted metric

In [ ]:
acc

In [ ]:
print(np.mean(acc, axis=0))
# accuracy metric (global)

### **Confusion Matrix**

In [ ]:
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}


model = AutoModelForSequenceClassification.from_pretrained("daveni/twitter-xlm-roberta-emotion-es", num_labels=6, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("daveni/twitter-xlm-roberta-emotion-es")
data_collator = DataCollatorWithPadding(tokenizer)


dataset = load_corpus_Salud(drop=0, n_labels=6, task="emotion")
train, test = train_test_split(dataset, stratify=dataset["label"], random_state=10)
train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

training_args = TrainingArguments(learning_rate=0.00001409, num_train_epochs=12, per_device_train_batch_size = 8, per_device_eval_batch_size=2,
                                  output_dir="best-sweep-2", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #for earlyStop

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[earlyStop]
)
trainer.train()
pred=trainer.predict(tokenized_datasets["test"])
logits=pred.predictions
prob=tf.nn.softmax(logits)
res=np.argmax(prob, axis=1) #predictions

In [ ]:
#Label Amor/Admiración is Love/admiration
#Label Gratitud is Gratitude
#Label Tristeza/Pena is Sadness
#Label Enfado/Desprecio/Burla is Anger/contempt/mockery
#Label Comprensión/Empatía/Identificación is Comprehension/empathy/identification
#Label Indeterminado is Neutral

confusionMatrix = confusion_matrix(tokenized_datasets["test"]["label"], res)

df_confusionMatrix = pd.DataFrame(confusionMatrix,
                                   index = ['Amor / Admiración', 'Gratitud', 'Tristeza / Pena', 'Enfado / Desprecio / Burla', 'Comprensión / Empatía / Identificación', 'Indiferente'],
                                    columns = ['Amor / Admiración', 'Gratitud', 'Tristeza / Pena', 'Enf / Despr / Burla', 'Compr / Emp / Indentif', 'Indiferente'])

plt.figure(figsize = (6, 4))
sns.heatmap(df_confusionMatrix, annot = True,fmt='g', annot_kws={"size": 14}, cmap = 'BuPu');

In [ ]:
id2label = {0: "joy", 1: "fear", 2: "sadness", 3:"anger", 4:"surprise", 5:"others"}
label2id = {"joy": 0, "fear": 1, "sadness": 2, "anger":3, "surprise":4, "others":5}


#"daveni/twitter-xlm-roberta-emotion-es"
#"pysentimiento/robertuito-emotion-analysis"
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-emotion-analysis", num_labels=6, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")
data_collator = DataCollatorWithPadding(tokenizer)


dataset = load_corpus_Salud(drop=0, n_labels=6, task="emotion")
train, test = train_test_split(dataset, stratify=dataset["label"], random_state=10)
train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)

training_args = TrainingArguments(learning_rate=0.00007557, num_train_epochs=6, per_device_train_batch_size = 16, per_device_eval_batch_size=16,
                                  output_dir="best-sweep-", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #for earlyStop

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[earlyStop]
)
trainer.train()
pred=trainer.predict(tokenized_datasets["test"])
logits=pred.predictions
prob=tf.nn.softmax(logits)
res=np.argmax(prob, axis=1) #predictions

In [ ]:
#Label Amor/Admiración is Love/admiration
#Label Gratitud is Gratitude
#Label Tristeza/Pena is Sadness
#Label Enfado/Desprecio/Burla is Anger/contempt/mockery
#Label Comprensión/Empatía/Identificación is Comprehension/empathy/identification
#Label Indeterminado is Neutral

confusionMatrix = confusion_matrix(tokenized_datasets["test"]["label"], res)

df_confusionMatrix = pd.DataFrame(confusionMatrix,
                                   index = ['Amor / Admiración', 'Gratitud', 'Tristeza / Pena', 'Enfado / Desprecio / Burla', 'Comprensión / Empatía / Identificación', 'Indiferente'],
                                    columns = ['Amor / Admiración', 'Gratitud', 'Tristeza / Pena', 'Enf / Despr / Burla', 'Compr / Emp / Indentif', 'Indiferente'])

plt.figure(figsize = (6, 4))
sns.heatmap(df_confusionMatrix, annot = True,fmt='g', annot_kws={"size": 14}, cmap = 'BuPu');

# **Example of predicting emotions of a text using the trained models**

In [ ]:
predict(name="robertuito", task="emotion",
        data=["te quiero", "te odio", "me das pena", "no tengo una opinion clara al respecto",
              "siento profundamente tu reciente pérdida",
              "soy vegano porque siento que debo hacer algo por el bienestar animal"])

   #Translation into English of data
   #data=["I love you", "I hate you", "I feel sorry for you", "I do not have a clear opinion on this issue",
    #          "I am deeply sorry for your recent loss",
     #         "I am vegan because I feel I must do something for animal welfare"])

# **Save models**

In [ ]:
dataset = load_corpus_Salud(0, n_labels=2, task="sentiment")
id2label = {0: "NEG",  1: "POS"}
label2id = {"NEG": 0, "POS": 1}


tokenizer = AutoTokenizer.from_pretrained('pysentimiento/roberta-es-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('pysentimiento/roberta-es-sentiment', num_labels=3, label2id = label2id, id2label=id2label, ignore_mismatched_sizes=True)
data_collator = DataCollatorWithPadding(tokenizer)

train, test = train_test_split(dataset, stratify=dataset["label"])

train.to_csv("corpus_train.csv", index=False)
test.to_csv("corpus_test.csv", index=False)
train_test = load_dataset("csv", data_files={"train":"./corpus_train.csv", "test":"./corpus_test.csv"})
tokenized_datasets = train_test.map(tokenize_function, batched=True)


training_args = TrainingArguments(learning_rate=0.00004863, num_train_epochs=13, per_device_train_batch_size = 64, per_device_eval_batch_size=128,
                                  output_dir="best-sweep-", evaluation_strategy="epoch", fp16=True, save_strategy="epoch", load_best_model_at_end = True, metric_for_best_model="accuracy") #necesario para earlyStop
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["test"],
      compute_metrics=compute_metrics,
      data_collator=data_collator,
      callbacks=[earlyStop]
)
trainer.train()

model.save_pretrained("./model_roberta_sentiment")
tokenizer.save_pretrained("./tok_roberta_sentiment")